In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical 

In [2]:
DATADIR = "D:/THESIS/Augmented Spectrogram/"

In [3]:
categories = os.listdir(DATADIR)
categories

['air_conditioner',
 'car_horn',
 'children_playing',
 'dog_bark',
 'drilling',
 'engine_idling',
 'gun_shot',
 'jackhammer',
 'siren',
 'street_music']

In [4]:
import tensorboard
tensorboard.__version__

'2.6.0'

In [5]:
data = []

def create_dataset():
    for category in categories:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = categories.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                #rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                new_array =img_array[34:252, 53:389]
                #print(new_array.shape)
                rsz=cv2.resize(new_array, (100, 100))
                #print(rsz.shape)
                data.append([rsz, class_num])  # add this to our training_data
                #plt.axis('off')
                #plt.imshow(rsz)
                #plt.savefig("ragresize.png")
                #break
            except Exception as e:  # in the interest in keeping the output clean...
                pass
    return data



In [6]:
%%time
data = create_dataset()

print(len(data))

100%|██████████████████████████████████████████████████████████████████████████████| 4000/4000 [01:09<00:00, 57.29it/s]

34928
Wall time: 10min 42s


In [7]:
X = []
Y = []

for features,label in data:
    X.append(features)
    Y.append(label)

In [8]:
import pickle

In [ ]:
pickle_out = open("X_DATA_augment_100x100_1.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("Y_DATA_augment_100x100_1.pickle","wb")
pickle.dump(Y,pickle_out)
pickle_out.close()


In [9]:
X = np.array(X)
Y = np.array(Y)

In [10]:
X.shape

(34928, 100, 100)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_data, X_test, Y_data, Y_test = train_test_split(X, Y,test_size=0.10, random_state = 41,stratify=Y)

In [13]:
X_train,X_val, Y_train, Y_val = train_test_split(X_data, Y_data,test_size=0.20, random_state = 41)

In [14]:
def normalizaze_data(X):
    X_data = []
    for data in X:
        data = data / 255
        X_data.append(data)
    return np.array(X_data)

In [ ]:
%%time
x_train = normalizaze_data(X_train)
x_val = normalizaze_data(X_val)
x_test = normalizaze_data(X_test)

In [ ]:
x_train = x_train.reshape(-1, 100, 100, 1)
x_val = x_val.reshape(-1, 100, 100, 1)
x_test = x_test.reshape(-1, 100, 100, 1)

In [ ]:
%%time
y_train =to_categorical(Y_train)
y_val = to_categorical(Y_val)
y_test =to_categorical(Y_test)

In [ ]:
Y_train[1]

In [ ]:
y_train[1]

In [ ]:
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_val.shape: {x_val.shape}")
print(f"y_val.shape: {y_val.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

In [ ]:
#importing other required libraries
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from sklearn.utils.multiclass import unique_labels
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras.utils import to_categorical
from keras import losses, models, optimizers
import time
from tensorflow.keras.layers import Input,Dense, Dropout, Activation, Flatten,concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.layers import (Convolution2D, Dense,Flatten, Dropout, GlobalAveragePooling2D,GlobalMaxPool2D, 
                          Input, MaxPool2D, concatenate)
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)

In [ ]:
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_val.shape: {x_val.shape}")
print(f"y_val.shape: {y_val.shape}")
print(f"x_val.shape: {x_test.shape}")
print(f"y_val.shape: {y_test.shape}")

In [ ]:
tf.optimizers.Adam()

In [ ]:
input_dim = x_train.shape[1:]
num_class = 10
MAX_PATIENT =5
MAX_EPOCHS = 150
batch_size = 64
lr = 0.001

In [ ]:
%%time
def CNN_Baseline(input_dim,num_class):
    inputs = Input(shape=input_dim)
    x = Convolution2D(64, (3, 3), activation = "relu") (inputs)
    x = MaxPool2D(pool_size=(2, 2)) (x)
    x =(Dropout(0.5))(x)
    
    x = Convolution2D(128, (3, 3), activation = "relu") (x)
    x = MaxPool2D(pool_size=(2, 2)) (x)
    x =(Dropout(0.5))(x)
   
    x = Convolution2D(128, (3, 3), activation = "relu") (x)
    x = MaxPool2D(pool_size=(2, 2)) (x)
    x =(Dropout(0.5))(x)
    
    x = Flatten()(x)
    
    x = Dense(100, activation = "relu") (x)
    x =(Dropout(0.5))(x)
    outputs = Dense(num_class, activation='softmax')(x)
    model = Model(inputs, outputs)
    
    return model 

model = CNN_Baseline(input_dim,num_class)


In [ ]:
# Define the Keras TensorBoard callback.
from datetime import datetime
import keras
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
opt = tf.optimizers.Adam(lr)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

checkpoint = ModelCheckpoint('Models/model_seed_41__.h5', monitor='val_loss', verbose=1, save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=MAX_PATIENT)
ReduceLR = ReduceLROnPlateau(monitpr = 'val_loss',factor=0.1,patience=3, verbose=1)
callbacks_list = [checkpoint,ReduceLR, early,tensorboard_callback]

print("[INFO] Training Start---------please Wait")
history_1 = model.fit(x_train, y_train, 
                    batch_size = batch_size,epochs = MAX_EPOCHS, 
                    validation_data =(x_val, y_val),
                    verbose = 1,callbacks = callbacks_list)
print("[INFO] Training Finished!!!")


In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file = 'model_plot.png', show_shapes = True, show_layer_names = True)

In [ ]:
%%time
score = model.evaluate(x_train, y_train, batch_size = 64)
print('training score: ', score)

score = model.evaluate(x_val, y_val, batch_size = 64)
print('validation score: ', score)

score = model.evaluate(x_test, y_test, batch_size = 64)
print('test score: ', score)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [ ]:
def plot_curve(history,fig_name):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    
    plt.figure(figsize=(10,10))
    plt.grid()
    plt.title('Training and validation Curve')
    plt.plot(epochs, acc, 'b',marker='o', label='Training acc')
    plt.plot(epochs, val_acc, 'r',marker='o', label='Validation acc')
    plt.legend()
    plt.plot(epochs, loss, 'm',marker='o', label='Training loss')
    plt.plot(epochs, val_loss, 'g',marker='o', label='Validation loss')
    plt.legend()
    plt.savefig(f"Figures/{fig_name}.png")
    plt.show()
    

In [ ]:
plot_curve(history_1,'fig_name_41')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_cm(y_true, y_pred, figsize=(10,10)):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    # cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm = pd.DataFrame(cm, index=[i for i in class_names],
                  columns = [i for i in class_names])
    
    cm.index.name = 'True label'
    cm.columns.name = 'Predicted label'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, annot=annot, fmt='', ax=ax)
    


In [ ]:
class_names = ['air_conditioner','car_horn','children_playing','dog_bark','drilling','engine_idling',
                      'gun_shot','jackhammer','siren','street_music']

In [ ]:
model_predictions = model.predict(x_test)
model_predicted_label = np.argmax(model_predictions, axis = 1)

In [ ]:
model_predicted_label[6]

In [ ]:
test_label = Y_test

In [ ]:
plot_cm(test_label, model_predicted_label)

In [ ]:
!pip install scikit-plot

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report

In [ ]:
y_true = test_label

In [ ]:
print(classification_report(y_true, model_predicted_label, target_names=class_names))

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score


target= class_names

# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (16, 16))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_true, model_predicted_label, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_true)
    y_test = lb.transform(y_true)
    y_pred = lb.transform(model_predicted_label)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)


print('ROC AUC score:', multiclass_roc_auc_score(y_true, model_predicted_label))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.show()